* The previous notebook began to run a little more slowly than desirable, so off we just into another one.
* Build a data modeling paradigm for practice.

In [15]:
from pandas import Series
import pandas as pd
import numpy as np
import scipy.stats

def beta_1_hat(dat):
    x_m = x_mean(dat)
    y_m = y_mean(dat)
    beta_1_hat_num = sum([(x - x_m)*(y - y_m) for (x,y) in dat.iteritems()])
    beta_1_hat_denom = sum([(d[0] - x_m)**2 for d in dat.iteritems()])
    beta_1_hat = beta_1_hat_num / beta_1_hat_denom
    return beta_1_hat

def beta_0_hat(dat):
    beta_0_hat = y_mean(dat) - beta_1_hat(dat)*x_mean(dat)
    return beta_0_hat

def n(dat):
    return len(dat)

def x_vals(dat):
    return np.array(dat.index)

def x_mean(dat):
    return np.mean(x_vals(dat))

def y_mean(dat):
    return dat.mean()

def sse(dat):
    return sum([(d[1] - (beta_0_hat + beta_1_hat*d[0]))**2 for d in dat.iteritems()])

def sst(dat):
    return sum([(d[1] - y_mean)**2 for d in dat.iteritems()])

def r_squared(dat):
    return 1 - sse(dat)/sst(dat)

def variance(dat)

variance_estimator = sse/(n-2)
standard_deviation_estimator = math.sqrt(variance_estimator)
beta_1_hat_standard_deviation_estimator = standard_deviation_estimator/math.sqrt(sum(x_vals**2) - (sum(x_vals)**2)/n)
beta_1_hat_standard_deviation_estimator

In [16]:
dat = Series({16.1:4.41,
              31.5:6.81,
              21.5:5.26,
              22.4:5.99,
              20.5:5.92,
              28.4:6.14, 
              30.3:6.84, 
              25.6:5.87, 
              32.7:7.03, 
              29.2:6.89, 
              34.7:7.87}
            )

In [17]:
y_mean(dat), x_mean(dat), beta_0_hat(dat), beta_1_hat(dat)

(6.275454545454545,
 26.627272727272725,
 2.2249345952600814,
 0.15211921970685938)